In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from mpl_toolkits.mplot3d import Axes3D

## Variáveis globais

In [ ]:
category_field = "client_id"

## Análise por categoria

In [152]:
base_ds = pd.read_csv(r"C:\laragon\www\tcc-gabriel\data\uci-online-retail-source\online_retail.csv", sep=",")
base_ds = base_ds.dropna()
encoder = OrdinalEncoder()
encoded_clients = encoder.fit_transform(
    base_ds['clientId'].values.reshape(-1, 1))
encoded_prods = encoder.fit_transform(
    base_ds['product_id'].values.reshape(-1, 1))
base_ds['clientId'] = encoded_clients
base_ds['product_id'] = encoded_prods
base_ds[:5]

,orderId,product_id,desc,avgQty,month,price,clientId,country
0,536365,3249.0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,"2,55",4048.0,United Kingdom
1,536365,2649.0,WHITE METAL LANTERN,6,12/1/10 8:26,"3,39",4048.0,United Kingdom
2,536365,2855.0,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,"2,75",4048.0,United Kingdom
3,536365,2803.0,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,"3,39",4048.0,United Kingdom
4,536365,2802.0,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,"3,39",4048.0,United Kingdom


In [ ]:
processed_amounts_ds = pd.read_csv(r"C:\Users\Gabriel Ullmann\Desktop\all_tcc_results\online_retail1\best-seller\online_retail_g_amounts.csv", sep=",")
processed_amounts_ds[:5]

In [ ]:
processed_ds = pd.read_csv(r"C:\Users\Gabriel Ullmann\Desktop\all_tcc_results\online_retail1\categ-cf\c_cbr.csv", sep=",")
processed_ds[:5]

In [ ]:
categories = pd.read_csv(r"C:\Users\Gabriel Ullmann\Desktop\all_tcc_results\online_retail1\categ-cf\best_selling_by_client.csv", sep=",")
categories = categories['country'].unique()
categories

In [ ]:
filtered_by_categ = []
col_clis = {}
for category in categories:
    filtered_by_categ = base_ds[(base_ds.country == category)]
    col_clis[category] = [];
    for row in filtered_by_categ.values:
        col_clis[category].append(int(row[6]))


In [ ]:
for category in categories:
    col_clis[category] = set(col_clis[category])
len(col_clis)

In [ ]:
def buildFilter(col_clis, dsname, fieldname, keyname):
    filterStr = "("
    ds_field = fieldname
    ccl = len(col_clis[keyname])
    count = 1
    for cli in col_clis[keyname]:
        filterStr += ds_field + "==" + str(cli)
        if count < ccl:
            filterStr += ") | ("
        count += 1
    filterStr += ")"
    return filterStr

In [137]:
for category in categories:
    npd = processed_ds[processed_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.displot(npd["rating"], kind="kde").set(title=category)
    sns_plot.savefig("temp_img/ratings_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: United Kingdom | Saved: Iceland | Saved: Finland | Saved: Italy | Saved: Norway | Saved: Bahrain | Saved: Spain | Saved: Portugal | Saved: Switzerland | Saved: Austria | Saved: Cyprus | Saved: Belgium | Saved: Unspecified | Saved: Denmark | Saved: Australia | Saved: France | Saved: Germany | Saved: RSA | Saved: Greece | Saved: Sweden | Saved: Israel | Saved: USA | Saved: Saudi Arabia | Saved: Poland | Saved: United Arab Emirates | Saved: Singapore | Saved: Japan | Saved: Netherlands | Saved: Lebanon | Saved: Brazil | Saved: Czech Republic | Saved: EIRE | Saved: Channel Islands | Saved: European Community | Saved: Lithuania | Saved: Canada | Saved: Malta | 

In [148]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.displot(npd["amount"], kde=False).set(title=category)
    plt.xlim(0, 500)
    sns_plot.savefig("temp_img/sales_volume_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: United Kingdom | Saved: Iceland | Saved: Finland | Saved: Italy | Saved: Norway | Saved: Bahrain | Saved: Spain | Saved: Portugal | Saved: Switzerland | Saved: Austria | Saved: Cyprus | Saved: Belgium | Saved: Unspecified | Saved: Denmark | Saved: Australia | Saved: France | Saved: Germany | Saved: RSA | Saved: Greece | Saved: Sweden | Saved: Israel | Saved: USA | Saved: Saudi Arabia | Saved: Poland | Saved: United Arab Emirates | Saved: Singapore | Saved: Japan | Saved: Netherlands | Saved: Lebanon | Saved: Brazil | Saved: Czech Republic | Saved: EIRE | Saved: Channel Islands | Saved: European Community | Saved: Lithuania | Saved: Canada | Saved: Malta | 

In [135]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.lmplot(x="product_id", y="amount", data=npd, fit_reg=False).set(title=category)
    sns_plot.savefig("temp_img/product_amount_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: United Kingdom | Saved: Iceland | Saved: Finland | Saved: Italy | Saved: Norway | Saved: Bahrain | Saved: Spain | Saved: Portugal | Saved: Switzerland | Saved: Austria | Saved: Cyprus | Saved: Belgium | Saved: Unspecified | Saved: Denmark | Saved: Australia | Saved: France | Saved: Germany | Saved: RSA | Saved: Greece | Saved: Sweden | Saved: Israel | Saved: USA | Saved: Saudi Arabia | Saved: Poland | Saved: United Arab Emirates | Saved: Singapore | Saved: Japan | Saved: Netherlands | Saved: Lebanon | Saved: Brazil | Saved: Czech Republic | Saved: EIRE | Saved: Channel Islands | Saved: European Community | Saved: Lithuania | Saved: Canada | Saved: Malta | 

In [160]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.lmplot(x="rating", y="amount", data=npd, fit_reg=False).set(title=category)
    sns_plot.savefig("temp_img/rating_amount_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: United Kingdom | Saved: Iceland | Saved: Finland | Saved: Italy | Saved: Norway | Saved: Bahrain | Saved: Spain | Saved: Portugal | Saved: Switzerland | Saved: Austria | Saved: Cyprus | Saved: Belgium | Saved: Unspecified | Saved: Denmark | Saved: Australia | Saved: France | Saved: Germany | Saved: RSA | Saved: Greece | Saved: Sweden | Saved: Israel | Saved: USA | Saved: Saudi Arabia | Saved: Poland | Saved: United Arab Emirates | Saved: Singapore | Saved: Japan | Saved: Netherlands | Saved: Lebanon | Saved: Brazil | Saved: Czech Republic | Saved: EIRE | Saved: Channel Islands | Saved: European Community | Saved: Lithuania | Saved: Canada | Saved: Malta | 

## Analisando produtos mais comuns em um intervalo
Conforme análise 2, categorias

In [157]:
un_prod = base_ds[(base_ds.product_id >= 1000) & (base_ds.product_id <= 2000)]['desc'].unique()
len(un_prod)

1088

In [161]:
un_prod[:5]

array(['SET 7 BABUSHKA NESTING BOXES', 'HAND WARMER UNION JACK',
       'HAND WARMER RED POLKA DOT', "POPPY'S PLAYHOUSE BEDROOM ",
       "POPPY'S PLAYHOUSE KITCHEN"], dtype=object)